In [1]:
!pip install transformers==4.3.2

In [2]:
import sys
import numpy as np
import pandas as pd
import os
os.chdir('..')
from GanBert import GanBert



In [3]:
def data_to_ganbert(data):    
    data = list(zip(data.iloc[:,0],data.iloc[:,1]))
    return data

In [4]:
data_path = 'data/medical'
files = os.listdir(data_path)
labeled_files = [data_path+"/"+file for file in files if "train_labeled" in file]

In [5]:
unlabeled = data_to_ganbert(pd.read_csv("data/train_unlabeled.csv"))
test = data_to_ganbert(pd.read_csv("data/test.csv"))

In [6]:
batch_size = 64
max_seq_length = 128
seed = 0
learning_rate = 5e-5
epochs=5
results=pd.DataFrame(columns=["n_per_class", "accuracy"])

In [7]:
for n_per_class in [5,10,25,50]:
    data_file = ""
    result = {"n_per_class":n_per_class}
    ganbert = GanBert(batch_size=batch_size,max_seq_length= max_seq_length,epochs = epochs,
                      learning_rate_discriminator = learning_rate,learning_rate_generator = learning_rate,
                      print_each_n_step = 100,random_state = seed)
    for file in labeled_files:
        if f"{data_path}/train_labeled_{n_per_class}.csv" == file:
            data_file = file
            break
    print(data_file)
    labeled = data_to_ganbert(pd.read_csv(data_file))
    performance = ganbert.train(labeled,unlabeled, test)
    result["accuracy"] = performance
    results = results.append(result,ignore_index=True)

There are 1 GPU(s) available.
We will use the GPU: Tesla K80
data/medical/train_labeled_5.csv

======== Epoch 1 / 5 ========
Training...

  Average training loss generetor: 0.605
  Average training loss discriminator: 2.493
  Training epcoh took: 0:04:55

Running Test...
  Accuracy: 0.011
  Test Loss: 2.582
  Test took: 0:00:06

======== Epoch 2 / 5 ========
Training...

  Average training loss generetor: 0.724
  Average training loss discriminator: 1.443
  Training epcoh took: 0:04:55

Running Test...
  Accuracy: 0.042
  Test Loss: 2.442
  Test took: 0:00:06

======== Epoch 3 / 5 ========
Training...

  Average training loss generetor: 0.717
  Average training loss discriminator: 0.922
  Training epcoh took: 0:04:55

Running Test...
  Accuracy: 0.136
  Test Loss: 2.332
  Test took: 0:00:06

======== Epoch 4 / 5 ========
Training...

  Average training loss generetor: 0.710
  Average training loss discriminator: 0.770
  Training epcoh took: 0:04:55

Running Test...
  Accuracy: 0.109
  

In [8]:
results

,n_per_class,accuracy
0,5.0,0.111607
1,10.0,0.000000
2,25.0,0.321429
3,50.0,0.392857


In [9]:
if not os.path.exists('results'):
      os.mkdir('results')
results.to_csv("results/GanBert_results_medical.csv")